In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from statsmodels.graphics.tsaplots import plot_acf

print(os.getcwd())

### Import data

In [ ]:
dataDir = "../data"
df = pd.read_csv("/content/ts_0011_sid6322070.csv")
                 #Weekend trends #ts_0109_sid6322115.csv")
                 #weekday trend #ts_0118_sid6322122.csv")
                 
                 #ts_0007_sid6321576.csv")
                 
                 #ts_0111_sid6322117.csv")


In [ ]:
df2 = df.drop(columns = ["Unnamed: 0", "sid", "uprns", "sensType"])
df2['datetime'] = pd.to_datetime(df['datetime'])
df2.head()

In [ ]:
# Add day and month name
df2['dayname'] = df2['datetime'].apply(lambda x: x.dayofweek)
df2['month'] = df2['datetime'].apply(lambda x: x.month)
df2['dayofyear'] = df2['datetime'].apply(lambda x: x.dayofyear)


In [ ]:
df.isnull().values.any()

In [ ]:
# Calulate overall mean
y_mean = df2['PM2_5'].mean()
df2['y_norm'] = df2['PM2_5']-y_mean


In [ ]:
# Plot weekday trend
days_dict = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
            }
df_day = df2.groupby("dayname", as_index = False).mean()
df_day = df_day.sort_values(by = ['dayname'], axis = 0, ascending = True)
df_day['dayname2'] = df_day['dayname'].apply(lambda x: days_dict[x])
plt.plot(df_day['dayname2'],df_day['y_norm'], color = 'dodgerblue')
plt.tick_params(axis='x', labelrotation = -45)

In [ ]:
month_dict = {
    1:"January",
    2:"February",
    3:"March",
    4:"April",
    5:"May",
    6:"June",
    7:"July",
    8:"August",
    9:"September",
    10:"October",
    11:"November",
    12:"December"
}

#Plot monthly trend
df_month = df2.groupby("month", as_index = False).mean()
df_month = df_month.sort_values(['month'],axis = 0, ascending = True)
df_month['month2'] = df_month['month'].apply(lambda x: month_dict[x])
plt.plot(df_month['month2'],df_month['y_norm'])


In [ ]:
#Plot monthly trend
df3 = df2.groupby("dayofyear").mean()
df3 = df3.sort_index(axis = 0, ascending = True)
plt.scatter(df3.index,df3['y_norm'])
import seaborn as sns
sns.relplot(
    data=df3, x="dayofyear", y="y_norm", kind="line"
)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
df_trends = df2[['datetime', 'PM2_5']].set_index('datetime')
res = seasonal_decompose(df_trends, model = "additive",periods = 36000)

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1,ylabel = "trend")
res.resid.plot(ax=ax2,ylabel = "seasoanlity")
res.seasonal.plot(ax=ax3,ylabel = "residual")
plt.show()

## Auto correlation and Partial corrlation plots

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df2['PM2_5'], lags=50, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df2['PM2_5'], lags=50, ax=ax2)
plt.show()

In [ ]:
plot_acf(res.seasonal)

In [ ]:
df_resampled = df_trends.resample('H').mean()
df_resampled = df_resampled.fillna(0)

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_resampled['PM2_5'], lags=50, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_resampled['PM2_5'], lags=50, ax=ax2)
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res = seasonal_decompose(df_resampled, model = "additive",period = 360)

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1,ylabel = "trend")
res.resid.plot(ax=ax2,ylabel = "seasoanlity")
res.seasonal.plot(ax=ax3,ylabel = "residual")
plt.show()#

In [ ]:
import altair as alt

df_alt = df_resampled.reset_index()
df_alt = df_alt.rename(columns = {'datetime':'ds',
                                 'PM2_5':'y'})
alt.data_transformers.enable('default', max_rows=None)
alt.Chart(df_alt).mark_line().encode(
    alt.X('ds:T'),
    alt.Y('y:Q',
         scale = alt.Scale(domain = (-50, 300)))
)

## Import weather data

In [ ]:
weather_2018 = pd.read_csv("../data/rawData/2018_hourly_weather.csv")
weather_2019 = pd.read_csv("../data/rawData/2019_hourly_weather.csv")
weather_2020 = pd.read_csv("../data/rawData/2020_hourly_weather.csv")

In [ ]:
df_weather = weather_2018.append(weather_2019, ignore_index = True)

In [ ]:
df_weather = df_weather.append(weather_2020, ignore_index = True)

In [ ]:
df_weather['ds'] = pd.to_datetime(df_weather['ds'])

### Join weather data to air quality data

In [ ]:
df_joined = df_alt.merge(df_weather, how = 'inner', on = 'ds')

In [ ]:
df_joined = df_joined[~df_joined['air_temperature'].isnull()]

In [ ]:
df_joined = pd.read_csv("/content/df_analysis.csv")

## Plotting Air Temp and PM2_5

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res_pm2_5 = seasonal_decompose(df_joined[['ds','y']].set_index('ds'), model = "additive",period = 360)
res_air_temp = seasonal_decompose(df_joined[['ds','air_temperature']].set_index('ds'), model = "additive",period = 360)

fig, ax = plt.subplots(figsize = (15, 8))

line1, = ax.plot(res_pm2_5.trend,label = "PM2_5", color = 'salmon')
ax.set_ylabel("PM2_5")
ax.tick_params(axis='y', colors='salmon')

ax2 = ax.twinx()

line2, = ax2.plot(res_air_temp.trend, label = "Air Temperature (C)", color = 'dodgerblue')
ax2.set_ylabel("Air Temperature (C)")
ax2.tick_params(axis='y', colors='dodgerblue')
ax2.spines['right'].set_color('dodgerblue')
ax2.spines['left'].set_color('salmon')
plt.legend(handles = [line1, line2])
ax.tick_params(axis='x', labelrotation = -45)
plt.rc('font', size = 15)

plt.show()

fig.savefig("plot.svg", dpi = 500)

## Testing for stationarity

In [ ]:
from statsmodels.tsa.stattools import adfuller
adfuller_test = adfuller(df_joined['y'], autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

In [ ]:
from statsmodels.tsa.stattools import adfuller
adfuller_test = adfuller(df_joined['air_temperature'], autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

p values suggest both variables are satationary so do not need to difference

## Getting best VAR model
https://towardsdatascience.com/multivariate-time-series-forecasting-456ace675971

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

df_var = df_joined.copy()
df_var.index = pd.DatetimeIndex(df_var.ds).to_period('H')
df_var = df_var.drop('ds', axis = 1)

In [ ]:
size = int(len(df_joined) * 0.8)
train = df_var[['y', 'air_temperature']].iloc[:size, :]
test = df_var[['y', 'air_temperature']].iloc[size:, :]           

In [ ]:
forecasting_model = VAR(train)

results_aic = []
for p in range(1, 10):
    results = forecasting_model.fit(p)
    results_aic.append(results.aic)

In [ ]:
import seaborn as sns
sns.set()
plt.plot(list(np.arange(1,500,1)), results_aic)
plt.xlabel("Order")
plt.ylabel("AIC")
plt.show()

In [ ]:
results_aic.index(min(results_aic))

In [ ]:
results = forecasting_model.fit(171)
results.summary()

In [ ]:
lagged_values = train.values[-171:]

forecast = pd.DataFrame(results.forecast(y = lagged_values, steps = 4378),
                       index = test.index, columns = ['y', 'air_temp'])

In [ ]:
forecast

## Time series forecasting
https://medium.com/analytics-vidhya/time-series-forecasting-a-complete-guide-d963142da33f

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from scipy.stats import variation

df_joined = pd.read_csv("/content/df_analysis.csv")

In [ ]:
%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error

In [ ]:
df = df_joined
df.shape

In [ ]:
df.info()

In [ ]:
df_pm25 = df[['ds', 'y']].set_index('ds').asfreq('H')

plt.figure(figsize=(18,7))
plt.plot(df_pm25, label='PM2_5', color = 'dodgerblue')
plt.legend(loc='best')
plt.xticks(rotation = 90,fontweight="bold")
plt.show()

In [ ]:
from pylab import rcParams
import statsmodels.api as sm

df_sd = df[['ds', 'y']].copy()
df_sd['ds'] = pd.to_datetime(df_sd['ds'])
df_sd = df_sd.set_index('ds')
df_sd = df_sd.resample('H').mean()
df_sd = df_sd.fillna(0)

In [ ]:
rcParams['figure.figsize'] = 12, 8
decomposition = sm.tsa.seasonal_decompose(df_sd, model='additive', period = 3600) # additive seasonal index
fig = decomposition.plot()
plt.show()

## Building and avaluating time series models

In [ ]:
train_len = int(len(df_sd)*0.8)
train = df_sd[0 : train_len]
test = df_sd[train_len : ]

### Simple time series methods

#### Naive method

In [ ]:
y_hat_naive = test.copy()
y_hat_naive['naive_forecast'] = train['y'][train_len-1]

In [ ]:
# Plot train, test and forecast

plt.figure(figsize=(12,4))
plt.plot(train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_naive['naive_forecast'], label='Naive forecast')
plt.legend(loc='best')
plt.title('Naive Method')
plt.show()

In [ ]:
# Calculate RMSE and MAPE

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test['y'], y_hat_naive['naive_forecast'])).round(2)
mape = np.round(np.mean(np.abs(test['y']-y_hat_naive['naive_forecast'])/test['y'])*100,2)

results = pd.DataFrame({'Method':['Naive method'], 'MAPE': [mape], 'RMSE': [rmse]})
results = results[['Method', 'RMSE', 'MAPE']]
results

#### Simple average method

In [ ]:
y_hat_avg = test.copy()
y_hat_avg['avg_forecast'] = train['y'].mean()

In [ ]:
# Plot train, test and forecast

plt.figure(figsize=(12,4))
plt.plot(train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_avg['avg_forecast'], label='Simple average forecast')
plt.legend(loc='best')
plt.title('Simple Average Method')
plt.show()

In [ ]:
# Calculate RMSE and MAPE

rmse = np.sqrt(mean_squared_error(test['y'], y_hat_avg['avg_forecast'])).round(2)
mape = np.round(np.mean(np.abs(test['y']-y_hat_avg['avg_forecast'])/test['y'])*100,2)

tempResults = pd.DataFrame({'Method':['Simple average method'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results

#### Simple moving average method

In [ ]:
y_hat_sma = df_sd.copy()
ma_window = 12
y_hat_sma['sma_forecast'] = df_sd['y'].rolling(ma_window).mean()
y_hat_sma['sma_forecast'][train_len:] = y_hat_sma['sma_forecast'][train_len-1]

In [ ]:
# Plot train, test and forecast

plt.figure(figsize=(12,4))
plt.plot(train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_sma['sma_forecast'], label='Simple moving average forecast')
plt.legend(loc='best')
plt.title('Simple Moving Average Method')
plt.show()

In [ ]:
# Calculate RMSE and MAPE

rmse = np.sqrt(mean_squared_error(test['y'], y_hat_sma['sma_forecast'][train_len:])).round(2)
mape = np.round(np.mean(np.abs(test['y']-y_hat_sma['sma_forecast'][train_len:])/test['y'])*100,2)

tempResults = pd.DataFrame({'Method':['Simple moving average forecast'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results

### Exponential Smoothing Techniques

#### Simple exponential smoothing

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
model = SimpleExpSmoothing(train['y'])
model_fit = model.fit(optimized=True)
model_fit.params
y_hat_ses = test.copy()
y_hat_ses['ses_forecast'] = model_fit.forecast(len(test))

In [ ]:
# Plot train, test, forecast

plt.figure(figsize=(12,4))
plt.plot(train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_ses['ses_forecast'], label='Simple exponential smoothing forecast')
plt.legend(loc='best')
plt.title('Simple Exponential Smoothing Method')
plt.show()

In [ ]:
# Calculate RMSE and MAPE

rmse = np.sqrt(mean_squared_error(test['y'], y_hat_ses['ses_forecast'])).round(2)
mape = np.round(np.mean(np.abs(test['y']-y_hat_ses['ses_forecast'])/test['y'])*100,2)

tempResults = pd.DataFrame({'Method':['Simple exponential smoothing forecast'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results

#### Holt's method with trend

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
model = ExponentialSmoothing(np.asarray(train['y']) ,seasonal_periods=12 ,trend='additive', seasonal=None)
model_fit = model.fit(optimized=True)
print(model_fit.params)
y_hat_holt = test.copy()
y_hat_holt['holt_forecast'] = model_fit.forecast(len(test))

In [ ]:
# Plot train, test, forecast

plt.figure(figsize=(12,4))
plt.plot( train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_holt['holt_forecast'], label='Holt\'s exponential smoothing forecast')
plt.legend(loc='best')
plt.title('Holt\'s Exponential Smoothing Method')
plt.show()

In [ ]:
# Calculate RMSE and MAPE

rmse = np.sqrt(mean_squared_error(test['y'], y_hat_holt['holt_forecast'])).round(2)
mape = np.round(np.mean(np.abs(test['y']-y_hat_holt['holt_forecast'])/test['y'])*100,2)

tempResults = pd.DataFrame({'Method':['Holt\'s exponential smoothing method'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results

#### Holt Winters' additive method with trend and seasonality

In [ ]:
y_hat_hwa = test.copy()
model = ExponentialSmoothing(np.asarray(train['y']) ,seasonal_periods=12 ,trend='add', seasonal='add')
model_fit = model.fit(optimized=True)
print(model_fit.params)
y_hat_hwa['hw_forecast'] = model_fit.forecast(len(test))

In [ ]:
# Plot train, test, forecast

plt.figure(figsize=(12,4))
plt.plot( train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_hwa['hw_forecast'], label='Holt Winters\'s additive forecast')
plt.legend(loc='best')
plt.title('Holt Winters\' Additive Method')
plt.show()

In [ ]:
# Calculate RMSE and MAPE

rmse = np.sqrt(mean_squared_error(test['y'], y_hat_hwa['hw_forecast'])).round(2)
mape = np.round(np.mean(np.abs(test['y']-y_hat_hwa['hw_forecast'])/test['y'])*100,2)

tempResults = pd.DataFrame({'Method':['Holt Winters\' additive method'], 'RMSE': [rmse],'MAPE': [mape] })
results = pd.concat([results, tempResults])
results = results[['Method', 'RMSE', 'MAPE']]
results

### Auto regressive methods

In [ ]:
# Check for sationarity using Augmented Dickey-Fuller (ADF) test

from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(df_sd['y'])

print('ADF Statistic: %f' % adf_test[0])
print('Critical Values @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' %adf_test[1])

Inference : p-value is less than 0.05. This means that the series is stationary.

In [ ]:
# Check for sationarity using Kwiatkowski-Phillips-Schmidt-Shin (KPSS) test

from statsmodels.tsa.stattools import kpss
kpss_test = kpss(df_sd['y'])

print('KPSS Statistic: %f' % kpss_test[0])
print('Critical Values @ 0.05: %.2f' % kpss_test[3]['5%'])
print('p-value: %f' % kpss_test[1])


Inference : p-value is greater than 0.05. This means that the series is not stationary.

### Auto regression method (AR)

In [ ]:
train_len = int(len(df_sd)-10)
train = df_sd[0 : train_len]
test = df_sd[train_len : ]

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(train, order=(1, 1, 10)) 
model_fit = model.fit()
print(model_fit.params)

In [ ]:
# Recover original time series

y_hat_ar = df_sd.copy()
y_hat_ar['ar_forecast'] = model_fit.predict(test.index.min(), test.index.max())


In [ ]:
# Plot train, test and forecast

plt.figure(figsize=(12,4))
#plt.plot(train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_ar['ar_forecast'][test.index.min():], label='Auto regression forecast')
plt.legend(loc='best')
plt.title('Auto Regression Method')
plt.show()

In [ ]:
y_hat_ar

### Seasonal auto regressive integrated moving average (SARIMA)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(train, order=(1, 1, 2), seasonal_order=(1, 1, 1, 12)) 
model_fit = model.fit()
print(model_fit.params)

In [ ]:
# Recover original time series forecast

y_hat_sarima = df_sd.copy()
y_hat_sarima['sarima_forecast'] = model_fit.predict(test.index.min(), test.index.max())

In [ ]:
# Plot train, test and forecast

plt.figure(figsize=(12,4))
#plt.plot(train['y'], label='Train')
plt.plot(test['y'], label='Test')
plt.plot(y_hat_sarima['sarima_forecast'][test.index.min():], label='SARIMA forecast')
plt.legend(loc='best')
plt.title('Seasonal autoregressive integrated moving average (SARIMA) method')
plt.show()

## Rolling Forecast

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from scipy.stats import variation
from pylab import rcParams
import statsmodels.api as sm

df_joined = pd.read_csv("/content/df_analysis.csv")

In [ ]:
%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error
df = df_joined
df.shape

In [ ]:
df_sd = df[['ds', 'y']].copy()
df_sd['ds'] = pd.to_datetime(df_sd['ds'])
df_sd = df_sd.set_index('ds')
df_sd = df_sd.resample('H').mean()
df_sd = df_sd.fillna(0)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

X = df_sd.squeeze()
size = int(len(df_sd) * 0.99)
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()


In [ ]:
for t in range(len(test)):
    model = ARIMA(history, order = (1, 0, 1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted = %f, expected = %f' % (yhat, obs))

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(history[-len(predictions):])
plt.plot(predictions)

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(history[-len(predictions):])
plt.plot(predictions)

## ARIMAX model with air temp FROM HERE

https://github.com/SimoAntikainen/sarimax-climate-change-forecast/blob/master/sarimax-climate-change-workfiles.ipynb

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from scipy.stats import variation
from pylab import rcParams
import statsmodels.api as sm
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import functions

%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

print(os.getcwd())

resamp_period = 'D'


/home/charlie/Documents/Uni/Exeter - Data Science/MTHM604_Tackling_Sustainability_Challenges/MTHM604_week_12/MTHM604_week_1/code


In [2]:
df = functions.read_data(file_directory = "../data/cleanData/df_analysis.csv", 
                    resample_period = resamp_period, 
                    resample_aggregate = 'mean')
df_sd = df
df_sd
df2 = functions.add_aggregate(df_sd, 'max', resamp_period)
df2

,y_mean,wind_direction_mean,wind_speed_mean,msl_pressure_mean,air_temperature_mean,rltv_hum_mean,stn_pres_mean,y_mean_max,wind_direction_mean_max,wind_speed_mean_max,msl_pressure_mean_max,air_temperature_mean_max,rltv_hum_mean_max,stn_pres_mean_max
ds,,,,,,,,,,,,,,
2018-01-01,2.922657,257.083333,15.000000,1004.420833,8.362500,75.675000,993.883333,2.922657,257.083333,15.000000,1004.420833,8.362500,75.675000,993.883333
2018-01-02,11.171889,227.500000,15.541667,1006.583333,10.408333,90.450000,996.100000,11.171889,227.500000,15.541667,1006.583333,10.408333,90.450000,996.100000
2018-01-03,22.949293,260.416667,21.416667,1004.495833,9.187500,75.345833,993.979167,22.949293,260.416667,21.416667,1004.495833,9.187500,75.345833,993.979167
2018-01-04,10.368371,252.916667,20.125000,998.366667,10.537500,85.329167,987.987500,10.368371,252.916667,20.125000,998.366667,10.537500,85.329167,987.987500
2018-01-05,19.582053,235.000000,8.416667,995.545833,6.108333,83.437500,984.995833,19.582053,235.000000,8.416667,995.545833,6.108333,83.437500,984.995833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,0.000000,281.666667,19.083333,980.750000,7.000000,76.295833,970.416667,0.000000,281.666667,19.083333,980.750000,7.000000,76.295833,970.416667
2020-12-28,0.000000,322.500000,21.000000,978.616667,6.800000,74.637500,968.295833,0.000000,322.500000,21.000000,978.616667,6.800000,74.637500,968.295833
2020-12-29,0.000000,310.416667,17.583333,995.166667,6.045833,76.650000,984.633333,0.000000,310.416667,17.583333,995.166667,6.045833,76.650000,984.633333


In [ ]:
tr_start, tr_end = '2018-01-01', '2020-05-31'
te_start, te_end = '2020-06-01', '2020-06-30'

In [ ]:
train = df_sd[tr_start : tr_end]
test = df_sd[te_start : te_end]
exog = df_sd[te_start : ]

In [ ]:
model = ARIMA(train.y, exog = train.air_temperature, order = (0, 0,1))
model_fitted = model.fit()
model_fitted.summary()

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fitted.resid)
fig, ax = plt.subplots(1,2, figsize = (15, 8))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
fig = plt.figure(figsize = (15, 8))
plt.plot(train['y'], label = "obs")
plt.plot(model_fitted.fittedvalues, label = "predicted")

# SARIMA Model

In [ ]:
sarimax = sm.tsa.statespace.SARIMAX(train.y,order=([1,0,1]),trend='c', seasonal_order=(1,0,1,12),
                                 enforce_stationarity=False, enforce_invertibility=False ).fit()
sarimax.summary()


In [ ]:
pred = sarimax.predict(tr_end,te_end)[1:]
print('SARIMA model MSE:{}'.format(mean_squared_error(test.y,pred)))

pd.DataFrame({'test':test.y.values.flatten(),'pred':pred}).plot();plt.show()



## SARIMAX Model with exogenous air temp data

In [ ]:
sarimax = sm.tsa.statespace.SARIMAX(train.y,order=(1,0,1),trend='c', seasonal_order=(1,0,1,12),
                                 enforce_stationarity=False, enforce_invertibility=False, exog=train.air_temperature).fit()
sarimax.summary()



In [ ]:
pred = sarimax.predict(tr_end,te_end,exog=test.air_temperature)[1:]
print('SARIMA model MSE:{}'.format(mean_squared_error(test.y,pred)))

pd.DataFrame({'test':test.y.values.flatten(),'pred':pred}).plot();plt.show()



In [ ]:
sarimax.plot_diagnostics(figsize=(15, 12))

# SARIMAX-model with all Exogenous data

In [ ]:
sarimax = sm.tsa.statespace.SARIMAX(train.y,order=(1,1,1),trend='c',seasonal_order=(1,1,1,12),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train[['air_temperature', 'wind_direction', 'wind_speed', 'msl_pressure', 'rltv_hum', 'stn_pres']]).fit()
sarimax.summary()


In [ ]:


pred = sarimax.predict(tr_end,te_end,exog=test[['air_temperature', 'wind_direction', 'wind_speed', 'msl_pressure', 'rltv_hum', 'stn_pres']])[1:]
print('SARIMA model MSE:{}'.format(mean_squared_error(test.y,pred)))

pd.DataFrame({'test':test.y.values.flatten(),'pred':pred}).plot();plt.show()



In [ ]:
## Removing non significant terms
sarimax = sm.tsa.statespace.SARIMAX(train.y,order=(1,1,1),trend='c',seasonal_order=(1,1,1,12),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train[['air_temperature', 'wind_direction', 'wind_speed', 'msl_pressure', 'stn_pres']]).fit()
sarimax.summary()


In [ ]:

pred = sarimax.predict(tr_end,te_end,exog=test[['air_temperature', 'wind_direction', 'wind_speed', 'msl_pressure', 'stn_pres']])[1:]
print('SARIMA model MSE:{}'.format(mean_squared_error(test.y,pred)))

pd.DataFrame({'test':test.y.values.flatten(),'pred':pred}).plot();plt.show()


In [ ]:
## Removing non significant terms
sarimax = sm.tsa.statespace.SARIMAX(train.y,order=(1,1,1),trend='c',seasonal_order=(1,1,1,12),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train[['air_temperature', 'wind_direction', 'wind_speed']]).fit()
sarimax.summary()


In [ ]:
pred = sarimax.predict(tr_end,te_end,exog=test[['air_temperature', 'wind_direction', 'wind_speed']])[1:]
print('SARIMA model MSE:{}'.format(mean_squared_error(test.y,pred)))

pd.DataFrame({'test':test.y.values.flatten(),'pred':pred}).plot();plt.show()


In [ ]:
## Removing ma term
sarimax = sm.tsa.statespace.SARIMAX(train.y,order=(1,1,0),trend='c',seasonal_order=(1,1,0,12),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train[['air_temperature', 'wind_direction', 'wind_speed']]).fit()
sarimax.summary()

In [ ]:
pred = sarimax.predict(tr_end,te_end,exog=test[['air_temperature', 'wind_direction', 'wind_speed']])[1:]
print('SARIMA model MSE:{}'.format(mean_squared_error(test.y,pred)))

pd.DataFrame({'test':test.y.values.flatten(),'pred':pred}).plot();plt.show()

In [ ]:
## Removing ma term
sarimax = sm.tsa.statespace.SARIMAX(train.y,order=(1,1,0),trend='c',seasonal_order=(1,1,0,12),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train[[ 'wind_speed']]).fit()
sarimax.summary()

In [ ]:
pred = sarimax.predict(tr_end,te_end,exog=test[['wind_speed']])[1:]
print('SARIMA model MSE:{}'.format(mean_squared_error(test.y,pred)))

pd.DataFrame({'test':test.y.values.flatten(),'pred':pred}).plot();plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res = seasonal_decompose(train.y, model = "additive")

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1,ylabel = "trend")
res.resid.plot(ax=ax2,ylabel = "residual")
res.seasonal.plot(ax=ax3,ylabel = "seasonality")
plt.show()

In [ ]:

fig,ax = plt.subplots(2,1,figsize=(20,10))
fig = sm.graphics.tsa.plot_acf(train.y.diff().dropna(), lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(train.y.diff().dropna(), lags=50, ax=ax[1])
plt.show()



# Final Predictions

In [ ]:


mod = sm.tsa.statespace.SARIMAX(train.y, order=(5,1,3),trend='c', seasonal_order=(5,1,3,12),
                                 enforce_stationarity=False, enforce_invertibility=False)
res = mod.fit()
res.summary()



In [ ]:


prediction1 = res.get_forecast(steps=214)



In [ ]:


prediction1.predicted_mean.tail()



In [ ]:


pred_ci1 = prediction1.conf_int()



In [ ]:
ax = train.y.plot(label='observed', figsize=(20, 15))
prediction1.predicted_mean.plot(ax=ax, label='forecast')
ax.fill_between(pred_ci1.index,
                pred_ci1.iloc[:, 0],
                pred_ci1.iloc[:, 1], color='k', alpha=.25)
ax.set_title('PM2.5 Forecast')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5')

plt.legend()
plt.show()

In [ ]:
ax = train.y.plot(label='observed', figsize=(20, 15))
prediction1.predicted_mean.plot(ax=ax, label='forecast')
ax.fill_between(pred_ci1.index,
                pred_ci1.iloc[:, 0],
                pred_ci1.iloc[:, 1], color='k', alpha=.25)
ax.set_title('PM2.5 Forecast')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5')

plt.legend()
plt.show()

# SARIMAX-prediction with air temp as exog

In [ ]:



mod = sm.tsa.statespace.SARIMAX(train.y,order=(5,1,3),trend='c', seasonal_order=(5,1,3,12),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train.air_temperature.values)
res = mod.fit()
res.summary()



In [ ]:

prediction2 = res.get_forecast(steps=214, exog=exog.air_temperature)

In [ ]:


prediction2.predicted_mean.head()



In [ ]:


pred_ci2 = prediction2.conf_int()



In [ ]:


ax = train.y.plot(label='observed', figsize=(20, 15))
prediction2.predicted_mean.plot(ax=ax, label='forecast')
test.y.plot(ax=ax, label = 'test')
ax.fill_between(pred_ci2.index,
                pred_ci2.iloc[:, 0],
                pred_ci2.iloc[:, 1], color='k', alpha=.25)
ax.set_title('PM2.5 Forecast')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5')

plt.legend()
plt.show()



# SARIMAX-prediction with all exogenous variable

In [ ]:

mod = sm.tsa.statespace.SARIMAX(train.y,order=(5,1,3),trend='c', seasonal_order=(5,1,3,12),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train[['air_temperature', 'wind_speed']])
res = mod.fit()
res.summary()


In [ ]:
prediction3 = res.get_forecast(steps=214, exog=exog[['air_temperature', 'wind_speed']].values)
pred_ci3 = prediction3.conf_int()
prediction3.predicted_mean.tail()

In [ ]:


ax = train.y.plot(label='observed', figsize=(20, 15))
prediction3.predicted_mean.plot(ax=ax, label='forecast')
test.y.plot(ax=ax, label = 'test')
ax.fill_between(pred_ci3.index,
                pred_ci3.iloc[:, 0],
                pred_ci3.iloc[:, 1], color='k', alpha=.25)
ax.set_title('PM2.5 Forecast')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5')

plt.legend()
plt.show()



In [ ]:


ax = train.y.plot(label='observed', figsize=(20, 15))
prediction1.predicted_mean.plot(ax=ax, label='PM2.5 only')
prediction2.predicted_mean.plot(ax=ax, label='PM2.5 with air temperature exogenous variable')
prediction3.predicted_mean.plot(ax=ax, label='PM2.5 with air temperature, wind direction, and wind speed as exogogenous variables')

ax.fill_between(pred_ci1.index,
                pred_ci1.iloc[:, 0],
                pred_ci1.iloc[:, 1], color='k', alpha=0.1)

ax.fill_between(pred_ci2.index,
                pred_ci2.iloc[:, 0],
                pred_ci2.iloc[:, 1], color='k', alpha=0.05)

ax.fill_between(pred_ci3.index,
                pred_ci3.iloc[:, 0],
                pred_ci3.iloc[:, 1], color='k', alpha=0.2)


ax.set_title('PM2.5 Forecast')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5')

plt.legend()
plt.show()



## Testing DON'T RUN

In [ ]:

mod = sm.tsa.statespace.SARIMAX(train.y,order=(5,1,3),trend='c', seasonal_order=(5,1,3,36),
                                 enforce_stationarity=False, enforce_invertibility=False,exog=train[['air_temperature', 'wind_speed']])
res = mod.fit()
res.summary()

In [ ]:
prediction3 = res.get_forecast(steps=214, exog=exog[['air_temperature', 'wind_speed']].values)
pred_ci3 = prediction3.conf_int()
prediction3.predicted_mean.tail()

In [ ]:
ax = train.y.plot(label='observed', figsize=(20, 15))
prediction3.predicted_mean.plot(ax=ax, label='forecast')
test.y.plot(ax=ax, label = 'test')
ax.fill_between(pred_ci3.index,
                pred_ci3.iloc[:, 0],
                pred_ci3.iloc[:, 1], color='k', alpha=.25)
ax.set_title('PM2.5 Forecast')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5')

plt.legend()
plt.show()



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from scipy.stats import variation
from pylab import rcParams
import statsmodels.api as sm
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

print(os.getcwd())

#df = pd.read_csv("/content/df_analysis.csv")
df = pd.read_csv("../data/cleanData/df_analysis.csv")

In [ ]:
df_sd = df.copy()
df_sd['ds'] = pd.to_datetime(df_sd['ds'])
df_sd = df_sd.set_index('ds')
df_sd = df_sd.resample('D').mean()
df_sd = df_sd.fillna(0)

In [ ]:
df_dict = {'mean': 'df_mean', 'sum': 'df_sum'}
for i in df_dict:
    print(i)
    
df_dict.keys()

In [ ]:
import pandas as pd
from functools import reduce
df = pd.DataFrame({'Date': [1,2,3,4], 'Value': [2,3,3,4]})
df_dict = {'df_1':df, 'df_2': df, 'df_3': df}
dfList
df_dict.keys()
#reduce(lambda df_dict.keys(): pd.merge(df_dict.keys(), on = 'Date'), df_dict.values())

In [ ]:
df2 = df.copy()
df2['ds'] = pd.to_datetime(df2['ds'])
df2 = df2.set_index('ds')
df2 = df2.resample('D').max()
df2 = df2.fillna(0)


In [ ]:
fig = plt.figure(figsize = (15,8))
plt.plot(df_sd.y.rolling(7, axis = 0).mean())

In [ ]:
fig = plt.figure(figsize = (15,8))
plt.plot(df_sd.y)

In [ ]:
fig = plt.figure(figsize = (15,8))
plt.plot(df2[df2.y < 150].y)